#### Initializing opencv haarcascade face detection network
https://github.com/opencv/opencv/tree/master/data/haarcascades
explained in: https://www.youtube.com/watch?v=7IFhsbfby9s&t=300s (or gitHub)

In [32]:
import cv2
import os

In [33]:
def convertImg(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def faceNetLocalize(img):
    img_cvt = convertImg(img)
    return faceNet.detectMultiScale3(img_cvt, scaleFactor=scaleFactor, minNeighbors=minNeighbors, minSize=minSize, outputRejectLevels = True)


In [34]:
scaleFactor = 1.1 #between 1.05 (quality) and 1.4 (speed) recommended (scale of the faces we search for)
minNeighbors = 4 #between 3 (quantity) and 6 (quality) recommended
minSize = (10, 10) #min size of a face in the picture
#maxSize = (240, 240) #max face size in picture

In [35]:
#TODO: testing with different model types
#for example: eye model + larger bounding box towards the bottom
faceNet = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#img = cv2.imread('../img/train/with_mask/0-with-mask.jpg')
#img = cv2.imread('testPic4.jpg')

#faceLocs, rejectLevels, confidences = faceNetLocalize(img)
#print(confidences)

In [36]:
#crop_imgs = []

#for (x, y, w, h) in faceLocs:
#    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
#    crop_imgs.append(img[y:y+h, x:x+w])
    

#cv2.imshow('img', img)
#cv2.waitKey(0)

#### Testing - Face detection

In [37]:
#TODO can we even use accuracy methods? should we just test freely with live feed and empirical testing?
#labels_path = os.path.join('..', 'img', 'test', 'labels')

#### Training - Mask classifier

foundation: https://www.tensorflow.org/tutorials/load_data/images

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import MaxPool2D, Conv2D, Input, Dense, Flatten, AveragePooling2D, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import Sequential, losses as lfs
from tensorflow.keras.callbacks import ModelCheckpoint

In [39]:
batch_size = 32
img_size = (180, 180)
img_size_vgg = (224, 224)
epochs = 10
checkpoint_path = "mask_model/weights.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
imgs_path = os.path.join('..', 'img')

correct_usage=  'correct usage: \n' + 'predict([path to image], \'category\' \n' + 'predict([path to image], \'probabilities\' \n' + 'predict([path to image], \'detection\' \n' + 'predict(\'live_detection\')'

In [40]:
train_ds = image_dataset_from_directory(imgs_path,  validation_split=0.2, subset="training",  seed=3, image_size=img_size,  batch_size=batch_size)
val_ds = image_dataset_from_directory(imgs_path,  validation_split=0.2, subset="validation",  seed=3, image_size=img_size,  batch_size=batch_size)
labels = train_ds.class_names
#train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
#val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 322 files belonging to 4 classes.
Using 258 files for training.
Found 322 files belonging to 4 classes.
Using 64 files for validation.


In [41]:
num_classes = 4

model = Sequential([ 
    Rescaling(1. /255),
    Conv2D(32, (3,3), activation='relu'),
    AveragePooling2D(pool_size=(7,7)),
    Flatten(name="flatten"),
    Dense(128, activation="relu"),
    Dropout(0.5),#drops small confidences
    Dense(num_classes, activation="softmax")
    ])

vgg_model = Sequential([
    Rescaling(1. /255),
    Conv2D(input_shape=(224,224,3), filters=64, kernel_size=(3,3), padding="same", activation="relu", strides=(1,1)), 
    Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"),
    MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"),
    MaxPool2D(pool_size=(2, 2), strides=(2)),
    Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"),
    MaxPool2D(pool_size=(2, 2), strides=(2)),
    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    MaxPool2D(pool_size=(2, 2), strides=(2)),
    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"),
    MaxPool2D(pool_size=(2, 2), strides=(2)),
    Flatten(),
    Dense(units=4096, activation="relu"),
    Dense(units=4096, activation="relu"),
    Dense(units=4, activation="softmax")
])
#model = vgg_model

model.compile(optimizer='adam', loss=lfs.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


about the VGG-model: https://neurohive.io/en/popular-networks/vgg16/

In [42]:
#Callback to save model's weights
#https://www.tensorflow.org/tutorials/keras/save_and_load
cp_callback = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose = 1 )

model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=[cp_callback])

Epoch 1/10


C:\Users\Julie\anaconda3\lib\site-packages\tensorflow\python\keras\backend.py:4942: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


9/9 [==============================] - 6s 254ms/step - loss: 1.7676 - accuracy: 0.2674 - val_loss: 1.1643 - val_accuracy: 0.4688

Epoch 00001: saving model to mask_model\weights.ckpt
Epoch 2/10
9/9 [==============================] - 5s 371ms/step - loss: 1.2599 - accuracy: 0.4884 - val_loss: 0.8793 - val_accuracy: 0.8281

Epoch 00002: saving model to mask_model\weights.ckpt
Epoch 3/10
9/9 [==============================] - 5s 349ms/step - loss: 0.8911 - accuracy: 0.6628 - val_loss: 0.5767 - val_accuracy: 0.7500

Epoch 00003: saving model to mask_model\weights.ckpt
Epoch 4/10
9/9 [==============================] - 6s 446ms/step - loss: 0.6829 - accuracy: 0.7054 - val_loss: 0.4517 - val_accuracy: 0.8750

Epoch 00004: saving model to mask_model\weights.ckpt
Epoch 5/10
9/9 [==============================] - 5s 367ms/step - loss: 0.6136 - accuracy: 0.7326 - val_loss: 0.4777 - val_accuracy: 0.7500

Epoch 00005: saving model to mask_model\weights.ckpt
Epoch 6/10
9/9 [=========================

#### Testing (mask): 

In [43]:
y_test = np.concatenate([y for _, y in val_ds], axis=0)
x_test = np.concatenate([x for x, _ in val_ds], axis=0)

y_pred_confidences = model.predict(x_test)
y_pred = [np.argmax(cs) for cs in y_pred_confidences]
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.31      0.31      0.31        13
           1       0.42      0.43      0.43        30
           2       0.00      0.00      0.00         4
           3       0.38      0.35      0.36        17

    accuracy                           0.36        64
   macro avg       0.28      0.27      0.27        64
weighted avg       0.36      0.36      0.36        64



##### Predictions:

In [44]:
#model = Sequential([
#    Rescaling(1. /255), 
#    Conv2D(32, 3, activation='relu'),
#    AveragePooling2D(pool_size=(7, 7)),
#    Flatten(name="flatten"),
#    Dense(128, activation="relu"),
#    Dropout(0.5),#drops small confidences
#    Dense(num_classes, activation="softmax")
#    ])
#model.load_weights(checkpoint_path)

In [45]:
def maskPredict(img):
    pred = model.predict(img[None])
    #TODO: richtiges mapping zwischen index/label?
    label_index = np.argmax(pred)
    return labels[label_index], pred[label_index]


#mode can be 'category', 'probabilities', 'detection', 'live_detection'
def predict(img_path, mode):
    #load_img
    img = cv2.imread(img_path)
    cv2.imshow('img', img)
    img = np.expand_dims(img, axis=0)
    
    if mode=='detection':
        detect(img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return
    if mode=='live_detection':
        live_det()
        return
        
    if mode=='category':
        label, confidence = maskPredict(img)
        return label
        
    if mode=='probabilities':
        return model.predict([img])

    else:
        print(correct_usage)

#def predict(mode):
#    if mode=='live_detection':
#        predict('', mode)
#    else:
#        print(correct_usage)

def detect(img):
    faceLocs, rejectLevels, confidences = faceNetLocalize(img)
        
    for (x, y, w, h) in faceLocs:
        #crop image and predict label of cropped image
        img_crop = img[y:y+h, x:x+w]
        label, confidence_mask = maskPredict(img_crop)
        #show label/ bounding box on image
        cv2.putText(img, f"{label}, confidence:{confidence_mask}", (x+w-30, y+h), cv2.FONT_HERSHEY_PLAIN, 1.0, cv2.CV_RGB(0,255,0), 2.0) 
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    cv2.imshow('live_output', img)

def live_det():
    #TODO: errorhandling for camera
    
    wait_time = 20 #time in ms to wait before refreshing feed
    camera = cv2.Videcapture(0) #Input value might differ on different systems
    
    while(True):
        _, img = camera.read()

        detect(img)

        #wait for ESC or q
        if (cv2.waitKey(wait_time) & 0xFF) in [27, ord('q')]: 
            break


    camera.release()
    return 'live_output'


In [47]:
img_path='../img/ffp2/IMG_1596_(657, 421, 1426, 1426).png'
#predict(img_path, 'category')
from tensorflow.keras.preprocessing import image
img = image.load_img(img_path, target_size = img_size)
img = image.img_to_array(img)
#img = np.expand_dims(img, axis = 0)
#np.shape(img[None])
maskPredict(img)

('ffp2',
 array([0.47268364, 0.0024329 , 0.37747186, 0.14741163], dtype=float32))